# First implementation of Gradient Descent

In [ ]:
import numpy as np

# Defining the sigmoid function for activations
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

# Derivative of the sigmoid function
def sigmoid_prime(x):
    """
    # Derivative of the sigmoid function
    """
    return sigmoid(x) * (1 - sigmoid(x))


# Input data
x = np.array([1, 2, 3, 4])
# Target
y = np.array(0.5)

# The learning rate which is used in the weight step equation
learnrate = 0.5

# Initial weights
w = np.array([0.5, -0.5, 0.3, 0.1])

### Calculate one gradient descent step for each weight
### Note: Some steps have been consolidated, so there are
###       fewer variable names than in the above sample code

# TODO: Calculate the node's linear combination of inputs and weights
# The linear combination performed by the node (h in f(h) and f'(h))
# np.dot multiplies two vectors and then sums up the resulting vector
h = np.dot(x, w)

# TODO:
# The neural network output (y-hat)
nn_output = sigmoid(h)

# TODO: Calculate the output error (y - y-hat) of the neural network
error = y - nn_output

# TODO: Calculate the error term
#       Remember, this requires the output gradient, which we haven't
#       specifically added a variable for.
# sigmoid_prime is the output gradient (f'(h))
# error term (lowercase delta)
error_term = sigmoid_prime(h) * error

# TODO: Calculate change in weights
# The gradient descent step 
del_w = learnrate * error_term * x

print('Neural Network output:')
print(nn_output)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)

# Second implementation of Gradient Descent
This time on a real dataset!

### Data Preparation

In [2]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('../data/nn_gradient_descent.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.iloc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

### Gradient Descent implementation

In [9]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.
def sigmoid_prime(output):
    return output * (1 - output)


def train(features, targets, epochs=1000, learnrate=0.1, random_state=42):
    # Use to same seed to make debugging easier
    np.random.seed(random_state)

    n_records, n_features = features.shape
    last_loss = None

    # Initialize weights
    weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

    # Neural Network hyperparameters
    epochs = epochs
    learnrate = learnrate

    for e in range(epochs):
        del_w = np.zeros(weights.shape)
        for x, y in zip(features.values, targets):
            # Loop through all records, x is the input, y is the target

            # Note: We haven't included the h variable from the previous
            #       lesson. You can add it if you want, or you can calculate
            #       the h together with the output

            # TODO: Calculate the output
            output = sigmoid(np.dot(x, weights))

            # TODO: Calculate the error
            error = y - output

            # TODO: Calculate the error term
            error_term = error * sigmoid_prime(output)

            # TODO: Calculate the change in weights for this sample
            #       and add it to the total weight change
            del_w += error_term * x

        # TODO: Update weights using the learning rate and the average change in weights
        weights += learnrate * del_w / n_records

        # Printing out the mean square error on the training set
        if e % (epochs / 10) == 0:
            out = sigmoid(np.dot(features, weights))
            loss = np.mean((out - targets) ** 2)
            if last_loss and last_loss < loss:
                print("Train loss: ", loss, "  WARNING - Loss Increasing")
            else:
                print("Train loss: ", loss)
            last_loss = loss
    
    return weights


def accuracy(features_test, targets_test, weights):
    # Calculate accuracy on test data
    tes_out = sigmoid(np.dot(features_test, weights))
    predictions = tes_out > 0.5
    accuracy = np.mean(predictions == targets_test)
    print("Prediction accuracy: {:.3f}".format(accuracy))

### Running the implementation with different settings

In [13]:
for epochs, learnrate in [[100, 0.5], [1000, 0.5], [5000, 0.5], [1000, 0.01]]:
    print(f'---Epochs: {epochs}, learning rate: {learnrate}---')
    %time weights = train(features, targets, epochs=epochs, learnrate=learnrate)
    accuracy(features_test, targets_test, weights)

---Epochs: 100, learning rate: 0.5---
Train loss:  0.2627609384996635
Train loss:  0.2505839260471835
Train loss:  0.24128247404777375
Train loss:  0.23395240246426385
Train loss:  0.22805408803546176
Train loss:  0.22325025799182846
Train loss:  0.2193105278798058
Train loss:  0.21606432596507505
Train loss:  0.21337877168621391
Train loss:  0.21114782763330286
Wall time: 498 ms
Prediction accuracy: 0.775
---Epochs: 1000, learning rate: 0.5---
Train loss:  0.2627609384996635
Train loss:  0.20928619409324875
Train loss:  0.20084292908073426
Train loss:  0.19862156475527873
Train loss:  0.19779851396686027
Train loss:  0.19742577912189863
Train loss:  0.1972350774624106
Train loss:  0.1971294562509248
Train loss:  0.19706766341315082
Train loss:  0.1970300580177737
Wall time: 4.35 s
Prediction accuracy: 0.725
---Epochs: 5000, learning rate: 0.5---
Train loss:  0.2627609384996635
Train loss:  0.19742577912189863
Train loss:  0.19700649668503784
Train loss:  0.19696789880063995
Train loss